In [1]:
import sklearn
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
scikit_version = sklearn.__version__

scikit_version

'0.24.1'

In [3]:
sentimental_data = pd.read_csv('../datasets/sentimental_analysis_data.csv', 
                               header=None, 
                               names=['Label', 'Text'], 
                               sep='\t')

sentimental_data.sample(10)

,Label,Text
2915,1,the story of Harry Potter is a deep and profou...
5013,0,MI: 2 was generally agreed to be a decent acti...
1170,1,I like MISSION IMPOSSIBLE but I hate Tom Cruis...
4999,0,After the god-awful “ Mission Impossible 2 ” f...
5362,0,I hate Harry Potter.
3346,1,Brokeback Mountain was so awesome.
4395,0,Da Vinci Code sucks.
4608,0,Da Vinci Code sucks be...
4005,0,"X3-good, Nacho Libre-sucked, Da Vinci Code-suc..."
4358,0,Da Vinci Code sucks be...


In [4]:
sentimental_data.shape

(6918, 2)

In [5]:
X = sentimental_data['Text']

Y = sentimental_data['Label']

In [6]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [7]:
x_train.shape, x_test.shape

((5534,), (1384,))

In [8]:
y_train.shape, y_test.shape

((5534,), (1384,))

In [9]:
tfidf_vect = TfidfVectorizer(max_features=15)

x_trans = tfidf_vect.fit_transform(x_train)

In [10]:
tfidf_vect

TfidfVectorizer(max_features=15)

In [11]:
print(x_trans[0:3])

  (0, 6)	0.7071067811865476
  (0, 9)	0.7071067811865476
  (1, 10)	0.7071067811865476
  (1, 2)	0.7071067811865476
  (2, 1)	0.5515286867287341
  (2, 14)	0.5497870249819378
  (2, 10)	0.44359347091497414
  (2, 2)	0.44359347091497414


In [12]:
x_trans.shape

(5534, 15)

In [13]:
classifier = LinearSVC(C=1.0, max_iter=1000, tol=1e-3)
linear_svc_model = classifier.fit(x_trans, y_train)

linear_svc_model

LinearSVC(tol=0.001)

In [14]:
x_test_trans = tfidf_vect.fit_transform(x_test)

In [15]:
x_test_trans.shape

(1384, 15)

In [16]:
y_pred = linear_svc_model.predict(x_test_trans)

y_pred

array([0, 1, 1, ..., 1, 0, 0], dtype=int64)

In [17]:
pred_results = pd.DataFrame({'y_test': y_test,
                             'y_pred': y_pred})

pred_results.sample(5)

,y_test,y_pred
2226,1,1
5576,0,0
4229,0,0
4681,0,0
737,1,1


In [18]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
accuracy

0.9046242774566474

In [19]:
text_clf_param = {}

text_clf_param['preprocessing'] = tfidf_vect
text_clf_param['model'] = linear_svc_model
text_clf_param['sklearn_version'] = scikit_version
text_clf_param['accuracy'] = accuracy

In [20]:
text_clf_param

{'preprocessing': TfidfVectorizer(max_features=15),
 'model': LinearSVC(tol=0.001),
 'sklearn_version': '0.24.1',
 'accuracy': 0.9046242774566474}

In [21]:
import joblib

In [22]:
filename = 'models/text_clf_checkpoint.joblib'

In [23]:
joblib.dump(text_clf_param, filename)

['models/text_clf_checkpoint.joblib']

In [24]:
clf_checkpoint = joblib.load(filename)

In [25]:
reloaded_vect = clf_checkpoint['preprocessing']

reloaded_vect

TfidfVectorizer(max_features=15)

In [26]:
clf_model = clf_checkpoint['model']

clf_model

LinearSVC(tol=0.001)

In [27]:
x_test_trans_new = reloaded_vect.fit_transform(x_test)

In [28]:
y_pred = clf_model.predict(x_test_trans_new)

y_pred

array([0, 1, 1, ..., 1, 0, 0], dtype=int64)

In [29]:
accuracy_score(y_test, y_pred)

0.9046242774566474

In [30]:
clf_checkpoint['accuracy']

0.9046242774566474

In [31]:
from sklearn.pipeline import Pipeline

In [32]:
clf_pipeline = Pipeline(steps=[('tfidf_vect', tfidf_vect), ('classifier', classifier)])

pipeline_model = clf_pipeline.fit(x_train, y_train)

In [33]:
pipeline_model

Pipeline(steps=[('tfidf_vect', TfidfVectorizer(max_features=15)),
                ('classifier', LinearSVC(tol=0.001))])

In [34]:
y_pred = pipeline_model.predict(x_test)

In [35]:
accuracy = accuracy_score(y_test, y_pred)

accuracy

0.9046242774566474

In [36]:
pipe_clf_param = {}

pipe_clf_param['pipeline_clf'] = pipeline_model
pipe_clf_param['sklearn_version'] = scikit_version
pipe_clf_param['accuracy'] = accuracy

In [37]:
pipe_clf_param

{'pipeline_clf': Pipeline(steps=[('tfidf_vect', TfidfVectorizer(max_features=15)),
                 ('classifier', LinearSVC(tol=0.001))]),
 'sklearn_version': '0.24.1',
 'accuracy': 0.9046242774566474}

In [38]:
filename = 'models/pipe_clf_checkpoint.joblib'

In [39]:
joblib.dump(pipe_clf_param, filename)

['models/pipe_clf_checkpoint.joblib']

In [40]:
pipe_clf_checkpoint = joblib.load(filename)

In [41]:
reloaded_pipeline = pipe_clf_checkpoint['pipeline_clf']

reloaded_pipeline

Pipeline(steps=[('tfidf_vect', TfidfVectorizer(max_features=15)),
                ('classifier', LinearSVC(tol=0.001))])

In [42]:
y_pred = reloaded_pipeline.predict(x_test)

In [43]:
accuracy_score(y_test, y_pred)

0.9046242774566474

In [44]:
pipe_clf_checkpoint['accuracy']

0.9046242774566474